In [25]:
import pandas as pd
import os
import math
from collections import defaultdict
from glicko2 import Player as Glicko2Player
import trueskill
import re
import numpy as np

pd.set_option('display.max_columns', None)

In [26]:
data = pd.DataFrame()
dataset_path = "../dataset"
for file in os.listdir(dataset_path):
    data_holder = pd.read_csv(f"../dataset/{file}")
    data = pd.concat([data, data_holder], ignore_index=True)


In [27]:
data['outcome'] = 1
data['tourney_date'] = pd.to_datetime(data['tourney_date'], format='%Y%m%d')
data = data.sort_values(by='tourney_date').reset_index(drop=True)
data['match_id'] = data.index


In [28]:
data

,tourney_id,tourney_name,surface,draw_size,tourney_level,tourney_date,match_num,winner_id,winner_seed,winner_entry,winner_name,winner_hand,winner_ht,winner_ioc,winner_age,loser_id,loser_seed,loser_entry,loser_name,loser_hand,loser_ht,loser_ioc,loser_age,score,best_of,round,minutes,w_ace,w_df,w_svpt,w_1stIn,w_1stWon,w_2ndWon,w_SvGms,w_bpSaved,w_bpFaced,l_ace,l_df,l_svpt,l_1stIn,l_1stWon,l_2ndWon,l_SvGms,l_bpSaved,l_bpFaced,winner_rank,winner_rank_points,loser_rank,loser_rank_points,outcome,match_id
0,1985-301,Auckland,Hard,32,A,1985-01-07,21,100744,6.0,NaN,Peter Doohan,R,190.0,AUS,23.6,100868,NaN,NaN,Craig A Miller,R,188.0,AUS,22.2,6-3 6-4,3,R16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,84.0,NaN,191.0,NaN,1,0
1,1985-301,Auckland,Hard,32,A,1985-01-07,23,100474,7.0,NaN,Chris Lewis,R,180.0,NZL,27.8,100519,NaN,NaN,Scott Mccain,R,180.0,USA,26.9,6-1 6-4,3,R16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,101.0,NaN,146.0,NaN,1,1
2,1985-301,Auckland,Hard,32,A,1985-01-07,22,100761,NaN,NaN,Danny Saltz,R,193.0,USA,23.4,100674,NaN,NaN,Jeff Turpin,R,188.0,USA,24.6,4-6 6-4 6-3,3,R16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,155.0,NaN,156.0,NaN,1,2
3,1985-301,Auckland,Hard,32,A,1985-01-07,1,100713,1.0,NaN,John Fitzgerald,R,183.0,AUS,24.0,100771,NaN,NaN,Kelvin Belcher,R,185.0,USA,23.2,2-6 6-3 6-4,3,R32,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,29.0,NaN,195.0,NaN,1,3
4,1985-301,Auckland,Hard,32,A,1985-01-07,8,100923,8.0,NaN,Wally Masur,R,180.0,AUS,21.6,100638,NaN,NaN,David Mustard,L,180.0,NZL,25.0,4-6 6-3 6-3,3,R32,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,106.0,NaN,168.0,NaN,1,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
129554,2024-7696,Next Gen Finals,Hard,8,F,2024-12-18,396,210530,5.0,NaN,Learner Tien,L,180.0,USA,19.0,209950,1.0,NaN,Arthur Fils,R,185.0,FRA,20.5,4-2 4-2 3-4(4) 4-3(5),5,RR,102.0,6.0,1.0,80.0,53.0,43.0,12.0,12.0,3.0,3.0,9.0,3.0,77.0,49.0,40.0,9.0,12.0,4.0,6.0,122.0,493.0,20.0,2355.0,1,129554
129555,2024-7696,Next Gen Finals,Hard,8,F,2024-12-18,395,211663,8.0,NaN,Joao Fonseca,R,185.0,BRA,18.3,209950,1.0,NaN,Arthur Fils,R,185.0,FRA,20.5,3-4(9) 4-2 4-1 1-4 4-1,5,RR,110.0,7.0,3.0,84.0,56.0,41.0,18.0,14.0,3.0,5.0,12.0,6.0,84.0,45.0,34.0,22.0,13.0,7.0,11.0,145.0,409.0,20.0,2355.0,1,129555
129556,2024-7696,Next Gen Finals,Hard,8,F,2024-12-18,394,210530,5.0,NaN,Learner Tien,L,180.0,USA,19.0,210150,3.0,NaN,Jakub Mensik,R,193.0,CZE,19.2,4-3(6) 4-3(3) 2-4 2-4 4-3(8),5,RR,137.0,1.0,1.0,113.0,76.0,51.0,21.0,15.0,6.0,9.0,24.0,2.0,93.0,61.0,51.0,18.0,15.0,0.0,1.0,122.0,493.0,48.0,1136.0,1,129556
129557,2024-7696,Next Gen Finals,Hard,8,F,2024-12-18,392,211663,8.0,NaN,Joao Fonseca,R,185.0,BRA,18.3,210530,5.0,NaN,Learner Tien,L,180.0,USA,19.0,4-0 4-0 1-4 4-2,5,RR,67.0,3.0,0.0,43.0,30.0,23.0,9.0,9.0,0.0,1.0,4.0,2.0,59.0,43.0,28.0,5.0,10.0,10.0,15.0,145.0,409.0,122.0,493.0,1,129557


In [29]:
def compute_rating_features(df):
    elo        = defaultdict(lambda: 1500)
    glicko     = defaultdict(lambda: Glicko2Player())
    ts_env     = trueskill.TrueSkill()
    ts_ratings = defaultdict(ts_env.Rating)

    rows = []

    for _, row in df.iterrows():
        A, B, result = row['winner_name'], row['loser_name'], row['outcome']

        rA_e, rB_e = elo[A], elo[B]
        pA,  pB    = glicko[A], glicko[B]
        tA,  tB    = ts_ratings[A], ts_ratings[B]

        exp_e = 1 / (1 + 10 ** ((rB_e - rA_e) / 400))
        q     = math.log(10) / 400
        g_phi = 1 / math.sqrt(1 + (3 * q**2 * pB.rd**2) / math.pi**2)
        exp_g = 1 / (1 + 10 ** (-g_phi * (pA.rating - pB.rating) / 400))

        quality = ts_env.quality_1vs1(tA, tB)

        rows.append({
            'tourney_date': row['tourney_date'],
            'winner_name':        A,  
            'loser_name':        B, 
            'winner_elo':          rA_e, 
            'loser_elo':          rB_e, 
            'elo_diff':       rA_e - rB_e, 
            'winner_elo_exp':       exp_e,
            'winner_glicko':       pA.rating,
            'winner_glicko_rd':    pA.rd,
            'loser_glicko':       pB.rating,
            'loser_glicko_rd':    pB.rd,
            'glicko_diff':    pA.rating - pB.rating, 
            'winner_glicko_exp':    exp_g,
            'winner_ts_mu':        tA.mu,
            'winner_ts_sigma':     tA.sigma,
            'loser_ts_mu':        tB.mu, 
            'loser_ts_sigma':     tB.sigma,
            'ts_quality':     quality,
            'match_id':       row['match_id']
        })

        K = 32
        elo[A] += K * (result - exp_e)
        elo[B] += K * ((1 - result) - (1 - exp_e))

        pA.update_player([pB.rating], [pB.rd], [result])
        pB.update_player([pA.rating], [pA.rd], [1 - result])

        if result == 1:
            new_tA, new_tB = ts_env.rate_1vs1(tA, tB)
        else:
            new_tB, new_tA = ts_env.rate_1vs1(tB, tA)
        ts_ratings[A], ts_ratings[B] = new_tA, new_tB

    return pd.DataFrame(rows)

In [30]:
df_features = compute_rating_features(data)

In [31]:
def extract_score_features(score):
    if pd.isna(score) or score.strip().upper() in {'W/O','WO','RET','RETIRE'}:
        return {
            'is_walkover': 1,
            'sets_A': np.nan, 'sets_B': np.nan,
            'games_A': np.nan, 'games_B': np.nan,
            'set_diff': np.nan, 'game_diff': np.nan,
            'n_tiebreaks': np.nan, 'pct_games_A': np.nan
        }
    pattern = r'(\d+)-(\d+)(?:\((\d+)\))?'
    raw = re.findall(pattern, score)
    sets = [(int(a), int(b), int(tb) if tb else 0) for a,b,tb in raw]

    sets_A = sum(1 for a,b,_ in sets if a > b)
    sets_B = sum(1 for a,b,_ in sets if b > a)
    games_A = sum(a for a,_,_ in sets)
    games_B = sum(b for _,b,_ in sets)
    n_tie = sum(1 for *_,tb in sets if tb > 0)
    total_games = games_A + games_B

    return {
        'is_walkover': 0,
        'sets_A': sets_A,
        'sets_B': sets_B,
        'games_A': games_A,
        'games_B': games_B,
        'set_diff': sets_A - sets_B,
        'game_diff': games_A - games_B,
        'n_tiebreaks': n_tie,
        'pct_games_A': games_A / total_games if total_games else np.nan
    }

score_feats = data['score'].apply(extract_score_features).apply(pd.Series)

df_feats = pd.concat([
    data[['tourney_date','winner_name','loser_name','score','outcome']],
    score_feats
], axis=1)

df_feats['winner_name'] = np.where(
    df_feats['outcome'] == 1,
    df_feats['winner_name'],
    df_feats['loser_name']
)
df_feats['loser_name'] = np.where(
    df_feats['outcome'] == 1,
    df_feats['loser_name'],
    df_feats['winner_name']
)

cols = [
    'tourney_date','winner_name','loser_name','score','is_walkover',
    'sets_A','sets_B','games_A','games_B','set_diff','game_diff',
    'n_tiebreaks','pct_games_A'
]
df_final = df_feats[cols].sort_values(['tourney_date']).reset_index(drop=True)
df_final['match_id'] = df_final.index

stats_cols = ['sets_A','sets_B','games_A','games_B','n_tiebreaks']

df_long = pd.concat([
    df_final[['match_id', 'tourney_date', 'winner_name'] + stats_cols].rename(columns={'winner_name':'player'}),
    df_final[['match_id', 'tourney_date', 'loser_name'] + stats_cols].rename(columns={'loser_name':'player'})
]).sort_values(['player', 'tourney_date', 'match_id']).reset_index(drop=True)

for stat in stats_cols:
    df_long[f'avg_{stat}'] = (
        df_long
        .groupby('player')[stat]
        .transform(lambda x: x.expanding().mean().shift())
    )

df_long_stats = df_long[['match_id','player'] + [f'avg_{c}' for c in stats_cols]]

df_model = df_final.merge(df_long_stats, 
                          left_on=['match_id','winner_name'], 
                          right_on=['match_id','player'], 
                          how='left').rename(columns={f'avg_{c}':f'winner_avg_{c}' for c in stats_cols})
df_model.drop(columns=['player'], inplace=True)

df_model = df_model.merge(df_long_stats, 
                          left_on=['match_id','loser_name'], 
                          right_on=['match_id','player'], 
                          how='left').rename(columns={f'avg_{c}':f'losser_avg_{c}' for c in stats_cols})
df_model.drop(columns=['player'], inplace=True)

to_drop = ['score','sets_A','sets_B','games_A','games_B','n_tiebreaks','is_walkover','set_diff','game_diff','pct_games_A']
df_model = df_model.drop(columns=to_drop)



In [32]:
data[['match_id',"tourney_date",'winner_name',"loser_name"]].sort_values(by='match_id',ascending=False)

,match_id,tourney_date,winner_name,loser_name
129558,129558,2024-12-18,Alex Michelsen,Juncheng Shang
129557,129557,2024-12-18,Joao Fonseca,Learner Tien
129556,129556,2024-12-18,Learner Tien,Jakub Mensik
129555,129555,2024-12-18,Joao Fonseca,Arthur Fils
129554,129554,2024-12-18,Learner Tien,Arthur Fils
...,...,...,...,...
4,4,1985-01-07,Wally Masur,David Mustard
3,3,1985-01-07,John Fitzgerald,Kelvin Belcher
2,2,1985-01-07,Danny Saltz,Jeff Turpin
1,1,1985-01-07,Chris Lewis,Scott Mccain


In [33]:
df_model[['match_id',"tourney_date",'winner_name',"loser_name"]].sort_values(by='match_id',ascending=False)

,match_id,tourney_date,winner_name,loser_name
129558,129558,2024-12-18,Alex Michelsen,Juncheng Shang
129557,129557,2024-12-18,Joao Fonseca,Learner Tien
129556,129556,2024-12-18,Joao Fonseca,Jakub Mensik
129555,129555,2024-12-18,Luca Van Assche,Nishesh Basavareddy
129554,129554,2024-12-18,Nishesh Basavareddy,Juncheng Shang
...,...,...,...,...
4,4,1985-01-07,Brad Drewett,Russell Barlow
3,3,1985-01-07,Chris Lewis,Wally Masur
2,2,1985-01-07,Howard Sands,Jonathan Canter
1,1,1985-01-07,Karl Meiler,Mark Wooldridge


In [34]:
df_final2 = data.merge(df_model, on = ['match_id',"tourney_date",'winner_name',"loser_name"], how = 'left')
df_final2 = df_final2.merge(df_features, on = ['match_id',"tourney_date",'winner_name',"loser_name"], how = 'left')

In [35]:
df_final2 = df_final2.drop(columns=
[
'match_id',
'outcome',
'loser_rank_points',
'loser_rank',
'winner_rank_points',
'winner_rank',
'score',
'loser_entry',
'loser_seed',
'loser_id',
'winner_entry',
'winner_seed',
'winner_id',
'match_num',
'tourney_id',
'draw_size',
'minutes',
'w_ace',
'w_df',
'w_svpt',
'w_1stIn',
'w_1stWon',
'w_2ndWon',
'w_SvGms',
'w_bpSaved',
'w_bpFaced',
'l_ace',
'l_df',
'l_svpt',
'l_1stIn',
'l_1stWon',
'l_2ndWon',
'l_SvGms',
'l_bpSaved',
'l_bpFaced'])

In [36]:
df_features.loc[(df_features.winner_name.str.contains("Chris Lewis")) | (df_features.loser_name.str.contains("Chris Lewis"))]

,tourney_date,winner_name,loser_name,winner_elo,loser_elo,elo_diff,winner_elo_exp,winner_glicko,winner_glicko_rd,loser_glicko,loser_glicko_rd,glicko_diff,winner_glicko_exp,winner_ts_mu,winner_ts_sigma,loser_ts_mu,loser_ts_sigma,ts_quality,match_id
1,1985-01-07,Chris Lewis,Scott Mccain,1500.000000,1500.000000,0.000000,0.500000,1500.000000,350.000000,1500.000000,350.000000,0.000000,0.500000,25.000000,8.333333,25.000000,8.333333,0.447214,1
9,1985-01-07,Chris Lewis,Steve Guy,1516.000000,1500.000000,16.000000,0.523010,1662.310895,290.318965,1500.000000,350.000000,162.310895,0.651386,29.395832,7.171476,25.000000,8.333333,0.443952,9
22,1985-01-07,Chris Lewis,Glenn Layendecker,1531.263693,1532.000000,-0.736307,0.498940,1750.542112,256.345172,1791.934554,247.463282,-41.392442,0.453289,31.957181,6.463874,33.072682,6.239836,0.545489,22
25,1985-01-07,Chris Lewis,Danny Saltz,1547.297601,1547.263693,0.033908,0.500049,1872.635926,222.103370,1861.194071,222.621454,11.441855,0.513445,35.500203,5.622785,35.191155,5.672383,0.593396,25
28,1985-01-07,Chris Lewis,Wally Masur,1563.296040,1562.527386,0.768653,0.501106,1961.474180,197.069959,1933.966385,199.706274,27.507795,0.533387,38.130908,5.006627,37.279627,5.149762,0.631617,28
529,1985-03-18,Jerome Vanier,Chris Lewis,1483.236175,1579.260642,-96.024467,0.365224,1319.390413,272.623697,2033.358611,178.002412,-713.968198,0.027162,18.036736,6.689953,40.263681,4.522819,0.049768,529
579,1985-03-25,Heinz Gunthardt,Chris Lewis,1456.538509,1558.947810,-102.409301,0.356746,1442.974309,169.566608,1925.825727,171.193392,-482.851418,0.080002,22.773813,3.403912,34.862038,4.084725,0.232779,579
679,1985-04-01,Tarik Benhabiles,Chris Lewis,1514.581223,1538.363685,-23.782462,0.465828,1585.503326,136.383467,1807.842273,164.109045,-222.338946,0.243212,24.621886,3.118893,30.576384,3.590285,0.571219,679
888,1985-04-29,Chris Lewis,Hans Gildemeister,1521.270169,1478.116971,43.153198,0.561785,1721.701149,152.574579,1517.062354,175.557582,204.638796,0.736731,28.135900,3.258106,24.650990,3.764806,0.689748,888
917,1985-04-29,Fernando Luna,Chris Lewis,1489.815382,1535.293047,-45.477666,0.434924,1556.301276,144.806260,1749.477340,144.848123,-193.176064,0.266909,24.083048,3.153361,28.942791,3.094504,0.643561,917


In [37]:
df_final2.loc[(df_final2.winner_name.str.contains("Chris Lewis")) | (df_final2.loser_name.str.contains("Chris Lewis"))]

,tourney_name,surface,tourney_level,tourney_date,winner_name,winner_hand,winner_ht,winner_ioc,winner_age,loser_name,loser_hand,loser_ht,loser_ioc,loser_age,best_of,round,winner_avg_sets_A,winner_avg_sets_B,winner_avg_games_A,winner_avg_games_B,winner_avg_n_tiebreaks,losser_avg_sets_A,losser_avg_sets_B,losser_avg_games_A,losser_avg_games_B,losser_avg_n_tiebreaks,winner_elo,loser_elo,elo_diff,winner_elo_exp,winner_glicko,winner_glicko_rd,loser_glicko,loser_glicko_rd,glicko_diff,winner_glicko_exp,winner_ts_mu,winner_ts_sigma,loser_ts_mu,loser_ts_sigma,ts_quality
1,Auckland,Hard,A,1985-01-07,Chris Lewis,R,180.0,NZL,27.8,Scott Mccain,R,180.0,USA,26.9,3,R16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1500.000000,1500.000000,0.000000,0.500000,1500.000000,350.000000,1500.000000,350.000000,0.000000,0.500000,25.000000,8.333333,25.000000,8.333333,0.447214
9,Auckland,Hard,A,1985-01-07,Chris Lewis,R,180.0,NZL,27.8,Steve Guy,R,188.0,NZL,25.8,3,R32,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1516.000000,1500.000000,16.000000,0.523010,1662.310895,290.318965,1500.000000,350.000000,162.310895,0.651386,29.395832,7.171476,25.000000,8.333333,0.443952
22,Auckland,Hard,A,1985-01-07,Chris Lewis,R,180.0,NZL,27.8,Glenn Layendecker,L,185.0,USA,23.6,3,QF,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1531.263693,1532.000000,-0.736307,0.498940,1750.542112,256.345172,1791.934554,247.463282,-41.392442,0.453289,31.957181,6.463874,33.072682,6.239836,0.545489
25,Auckland,Hard,A,1985-01-07,Chris Lewis,R,180.0,NZL,27.8,Danny Saltz,R,193.0,USA,23.4,3,SF,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1547.297601,1547.263693,0.033908,0.500049,1872.635926,222.103370,1861.194071,222.621454,11.441855,0.513445,35.500203,5.622785,35.191155,5.672383,0.593396
28,Auckland,Hard,A,1985-01-07,Chris Lewis,R,180.0,NZL,27.8,Wally Masur,R,180.0,AUS,21.6,5,F,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1563.296040,1562.527386,0.768653,0.501106,1961.474180,197.069959,1933.966385,199.706274,27.507795,0.533387,38.130908,5.006627,37.279627,5.149762,0.631617
529,Nancy,Hard,A,1985-03-18,Jerome Vanier,L,175.0,FRA,27.3,Chris Lewis,R,180.0,NZL,28.0,3,R32,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1483.236175,1579.260642,-96.024467,0.365224,1319.390413,272.623697,2033.358611,178.002412,-713.968198,0.027162,18.036736,6.689953,40.263681,4.522819,0.049768
579,Milan,Carpet,A,1985-03-25,Heinz Gunthardt,R,180.0,SUI,26.1,Chris Lewis,R,180.0,NZL,28.0,3,R32,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1456.538509,1558.947810,-102.409301,0.356746,1442.974309,169.566608,1925.825727,171.193392,-482.851418,0.080002,22.773813,3.403912,34.862038,4.084725,0.232779
679,Monte Carlo,Clay,A,1985-04-01,Tarik Benhabiles,R,175.0,FRA,20.1,Chris Lewis,R,180.0,NZL,28.0,3,R64,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1514.581223,1538.363685,-23.782462,0.465828,1585.503326,136.383467,1807.842273,164.109045,-222.338946,0.243212,24.621886,3.118893,30.576384,3.590285,0.571219
888,Hamburg,Clay,A,1985-04-29,Chris Lewis,R,180.0,NZL,28.1,Hans Gildemeister,R,183.0,CHI,29.2,3,R64,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1521.270169,1478.116971,43.153198,0.561785,1721.701149,152.574579,1517.062354,175.557582,204.638796,0.736731,28.135900,3.258106,24.650990,3.764806,0.689748
917,Hamburg,Clay,A,1985-04-29,Fernando Luna,R,175.0,ESP,27.0,Chris Lewis,R,180.0,NZL,28.1,3,R32,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1489.815382,1535.293047,-45.477666,0.434924,1556.301276,144.806260,1749.477340,144.848123,-193.176064,0.266909,24.083048,3.153361,28.942791,3.094504,0.643561


# look into these coll to not data leak to the model.
minutes	w_ace	w_df	w_svpt	w_1stIn	w_1stWon	w_2ndWon	w_SvGms	w_bpSaved	w_bpFaced	l_ace	l_df	l_svpt	l_1stIn	l_1stWon	l_2ndWon	l_SvGms	l_bpSaved	l_bpFaced

In [38]:
data

,tourney_id,tourney_name,surface,draw_size,tourney_level,tourney_date,match_num,winner_id,winner_seed,winner_entry,winner_name,winner_hand,winner_ht,winner_ioc,winner_age,loser_id,loser_seed,loser_entry,loser_name,loser_hand,loser_ht,loser_ioc,loser_age,score,best_of,round,minutes,w_ace,w_df,w_svpt,w_1stIn,w_1stWon,w_2ndWon,w_SvGms,w_bpSaved,w_bpFaced,l_ace,l_df,l_svpt,l_1stIn,l_1stWon,l_2ndWon,l_SvGms,l_bpSaved,l_bpFaced,winner_rank,winner_rank_points,loser_rank,loser_rank_points,outcome,match_id
0,1985-301,Auckland,Hard,32,A,1985-01-07,21,100744,6.0,NaN,Peter Doohan,R,190.0,AUS,23.6,100868,NaN,NaN,Craig A Miller,R,188.0,AUS,22.2,6-3 6-4,3,R16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,84.0,NaN,191.0,NaN,1,0
1,1985-301,Auckland,Hard,32,A,1985-01-07,23,100474,7.0,NaN,Chris Lewis,R,180.0,NZL,27.8,100519,NaN,NaN,Scott Mccain,R,180.0,USA,26.9,6-1 6-4,3,R16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,101.0,NaN,146.0,NaN,1,1
2,1985-301,Auckland,Hard,32,A,1985-01-07,22,100761,NaN,NaN,Danny Saltz,R,193.0,USA,23.4,100674,NaN,NaN,Jeff Turpin,R,188.0,USA,24.6,4-6 6-4 6-3,3,R16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,155.0,NaN,156.0,NaN,1,2
3,1985-301,Auckland,Hard,32,A,1985-01-07,1,100713,1.0,NaN,John Fitzgerald,R,183.0,AUS,24.0,100771,NaN,NaN,Kelvin Belcher,R,185.0,USA,23.2,2-6 6-3 6-4,3,R32,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,29.0,NaN,195.0,NaN,1,3
4,1985-301,Auckland,Hard,32,A,1985-01-07,8,100923,8.0,NaN,Wally Masur,R,180.0,AUS,21.6,100638,NaN,NaN,David Mustard,L,180.0,NZL,25.0,4-6 6-3 6-3,3,R32,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,106.0,NaN,168.0,NaN,1,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
129554,2024-7696,Next Gen Finals,Hard,8,F,2024-12-18,396,210530,5.0,NaN,Learner Tien,L,180.0,USA,19.0,209950,1.0,NaN,Arthur Fils,R,185.0,FRA,20.5,4-2 4-2 3-4(4) 4-3(5),5,RR,102.0,6.0,1.0,80.0,53.0,43.0,12.0,12.0,3.0,3.0,9.0,3.0,77.0,49.0,40.0,9.0,12.0,4.0,6.0,122.0,493.0,20.0,2355.0,1,129554
129555,2024-7696,Next Gen Finals,Hard,8,F,2024-12-18,395,211663,8.0,NaN,Joao Fonseca,R,185.0,BRA,18.3,209950,1.0,NaN,Arthur Fils,R,185.0,FRA,20.5,3-4(9) 4-2 4-1 1-4 4-1,5,RR,110.0,7.0,3.0,84.0,56.0,41.0,18.0,14.0,3.0,5.0,12.0,6.0,84.0,45.0,34.0,22.0,13.0,7.0,11.0,145.0,409.0,20.0,2355.0,1,129555
129556,2024-7696,Next Gen Finals,Hard,8,F,2024-12-18,394,210530,5.0,NaN,Learner Tien,L,180.0,USA,19.0,210150,3.0,NaN,Jakub Mensik,R,193.0,CZE,19.2,4-3(6) 4-3(3) 2-4 2-4 4-3(8),5,RR,137.0,1.0,1.0,113.0,76.0,51.0,21.0,15.0,6.0,9.0,24.0,2.0,93.0,61.0,51.0,18.0,15.0,0.0,1.0,122.0,493.0,48.0,1136.0,1,129556
129557,2024-7696,Next Gen Finals,Hard,8,F,2024-12-18,392,211663,8.0,NaN,Joao Fonseca,R,185.0,BRA,18.3,210530,5.0,NaN,Learner Tien,L,180.0,USA,19.0,4-0 4-0 1-4 4-2,5,RR,67.0,3.0,0.0,43.0,30.0,23.0,9.0,9.0,0.0,1.0,4.0,2.0,59.0,43.0,28.0,5.0,10.0,10.0,15.0,145.0,409.0,122.0,493.0,1,129557
